## Install library

#### Projeto localizado no site 
https://archive.ics.uci.edu/dataset/45/heart+disease


#### Introdução pyspark
https://blog.devgenius.io/beginners-introduction-to-big-data-in-pyspark-9931d919521c

#### Transformações em pyspark feature engineering
https://medium.com/@nutanbhogendrasharma/feature-transformer-vectorassembler-in-pyspark-ml-feature-part-3-b3c2c3c93ee9

#### manipulação e combinação de tatasets com pyspark (joins)
https://datalivre.medium.com/joins-em-pyspark-3c1d2773eeb1

##### Descrição dos campos:
Only 14 attributes used:
      1. #3  (age
          3 age: age in years 
      2. #4  (sex)
          4 sex: sex (1 = male; 0 = female)  
      3. #9  (cp) 
          9 cp: chest pain type
        -- Value 1: typical angina
        -- Value 2: atypical angina
        -- Value 3: non-anginal pain
        -- Value 4: asymptomatic   
      4. #10 (t
          10 trestbps: resting blood pressure (in mm Hg on admission to the hospital)stbps)  
      5. #1
          12 chol: serum cholestoral in mg/dlol)      
      6.
          16 fbs: (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)bs)       
      7. #19 (
          19 restecg: resting electrocardiographic results
        -- Value 0: normal
        -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
        -- Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteriarestecg)   
      8.
          32 thalach: maximum heart rate achieved32 (thalach)   
      9
          38 exang: exercise induced angina (1 = yes; 0 = no)#38 (ang)     
      10
          40 oldpeak = ST depression induced by exercise relative to rest #40 ldpeak)   
      1
          41 slope: the slope of the peak exercise ST segment
        -- Value 1: upsloping
        -- Value 2: flat
        -- Value 3: downsloping. 1 (slope)     

          44 ca: number of major vessels (0-3) colored by flourosopy  12.44 (ca)        
          51 thal: 3 = normal; 6 = fixed defect; 7 = reversable defect   13#51 (thal)      
      14. #58 (num)       (
          58 num: diagnosis of heart disease (angiographic disease status)
        -- Value 0: < 50% diameter narrowing
        -- Value 1: > 50% diameter narrowing
        (in any major vessel: attributes 59 through 68 are vessels)the predicted attribute)


In [1]:
!pip install numpy==1.23.*

In [2]:
import numpy

In [3]:
numpy.__version__

'1.23.5'

In [4]:
!pip install -q pyspark
!pip install -q handyspark

In [5]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import urllib.request
from pyspark.sql.functions import col, count, isnan, when


In [6]:
#Create Session
conf = SparkConf() \
    .set("spark.executor.instances", "2") \
    .set("spark.executor.memory", "2g") \
    .set("spark.driver.memory", "2g") \
    .setAppName("MeuAPP")

spark = SparkSession.builder \
        .config(conf=conf) \
        .getOrCreate()

# Obter o contexto Spark da sessão
sc = spark.sparkContext

# # Parar a sessão Spark quando não for mais necessária
# spark.stop()

# # Parar o contexto Spark quando não for mais necessário
# sc.stop()

In [7]:
# Imprimir informações sobre a sessão Spark
print("Session Spark:")
print(" - ID da aplicação:", spark.sparkContext.applicationId)
print(" - Nome da aplicação:", spark.sparkContext.appName)

# Imprimir informações sobre o contexto Spark
print("\nContexto Spark:")
print(" - Versão do Spark:", sc.version)
print(" - Modo de execução:", sc.master)

# Print the Python version of SparkContext
print("\nThe Python version of Spark Context in the PySpark shell is", sc.pythonVer)

Session Spark:
 - ID da aplicação: local-1707790600046
 - Nome da aplicação: MeuAPP

Contexto Spark:
 - Versão do Spark: 3.5.0
 - Modo de execução: local[*]

The Python version of Spark Context in the PySpark shell is 3.11


In [8]:
spark

In [9]:
#Link para o conjunto de dados Iris
link_dados = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
local_path = "./data/raw/heard_disease_raw_data.csv"  # Caminho local onde o arquivo será salvo

# Baixar o arquivo CSV do URL
urllib.request.urlretrieve(link_dados, local_path)

# Nomes colunas
nomes_colunas=["age", "sex", "cp", "trestbps", "chol", "fbs", 
                "restecg", "thalach", "exang", "oldpeak",
                "slope","ca","thal","num"
              ]

# Carregar os dados como um DataFrame Spark
df = spark.read.csv(local_path, header=None, inferSchema=True)

# Atribuir nomes às colunas
df = df.toDF(*nomes_colunas)

# Exibir o esquema dos dados
print("Esquema dos Dados:")
df.printSchema()

# Exibir a quantidade de colunas e linhas
print("Shape of the dataset: ", (df.count(), len(df.columns)))

# Exibir as primeiras 5 linhas dos dados
print("\nPrimeiras 5 linhas dos Dados:")
df.show(5)

Esquema dos Dados:
root
 |-- age: double (nullable = true)
 |-- sex: double (nullable = true)
 |-- cp: double (nullable = true)
 |-- trestbps: double (nullable = true)
 |-- chol: double (nullable = true)
 |-- fbs: double (nullable = true)
 |-- restecg: double (nullable = true)
 |-- thalach: double (nullable = true)
 |-- exang: double (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slope: double (nullable = true)
 |-- ca: string (nullable = true)
 |-- thal: string (nullable = true)
 |-- num: integer (nullable = true)

Shape of the dataset:  (303, 14)

Primeiras 5 linhas dos Dados:
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
| age|sex| cp|trestbps| chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|num|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
|63.0|1.0|1.0|   145.0|233.0|1.0|    2.0|  150.0|  0.0|    2.3|  3.0|0.0| 6.0|  0|
|67.0|1.0|4.0|   160.0|286.0|0.0|    2.0|  108.0|  1.0|    1.5|  2.

In [10]:
df_describe = df.describe()

In [11]:
df_describe.show()

+-------+-----------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+
|summary|              age|                sex|                cp|          trestbps|              chol|               fbs|           restecg|           thalach|              exang|           oldpeak|             slope|                ca|              thal|               num|
+-------+-----------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+
|  count|              303|                303|               303|               303|               303|               303|               303|               303|        

## Data Preprocessing

In [12]:
df.groupBy('thal').agg(count("*").alias("count")).show()

+----+-----+
|thal|count|
+----+-----+
| 6.0|   18|
| 7.0|  117|
|   ?|    2|
| 3.0|  166|
+----+-----+



In [13]:
null_df = df.select([count(when(
                                col(c).contains('None') | \
                                col(c).contains('NULL') | \
                                (col(c) == '') | \
                                col(c).isNull() | \
                                isnan(col(c)), c ))
           .alias(c)
           for c in df.columns])

null_df.show()

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+---+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|num|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+---+
|  0|  0|  0|       0|   0|  0|      0|      0|    0|      0|    0|  0|   0|  0|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+---+



In [14]:
df.select("*").filter(col('age') == 50).show()

+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
| age|sex| cp|trestbps| chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|num|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
|50.0|0.0|3.0|   120.0|219.0|0.0|    0.0|  158.0|  0.0|    1.6|  2.0|0.0| 3.0|  0|
|50.0|1.0|4.0|   150.0|243.0|0.0|    2.0|  128.0|  0.0|    2.6|  2.0|0.0| 7.0|  4|
|50.0|1.0|3.0|   140.0|233.0|0.0|    0.0|  163.0|  0.0|    0.6|  2.0|1.0| 7.0|  1|
|50.0|1.0|3.0|   129.0|196.0|0.0|    0.0|  163.0|  0.0|    0.0|  1.0|0.0| 3.0|  0|
|50.0|0.0|2.0|   120.0|244.0|0.0|    0.0|  162.0|  0.0|    1.1|  1.0|0.0| 3.0|  0|
|50.0|0.0|4.0|   110.0|254.0|0.0|    2.0|  159.0|  0.0|    0.0|  1.0|0.0| 3.0|  0|
|50.0|1.0|4.0|   144.0|200.0|0.0|    2.0|  126.0|  1.0|    0.9|  2.0|0.0| 7.0|  3|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+



## Exploratory Data Analysis

In [15]:
###########################################
# Trabalhando com RDD 
##########################################

# Verification of the actuality dataset
type(df)

pyspark.sql.dataframe.DataFrame

In [16]:
# Create a fileRDD from filepath
fileRDD = sc.textFile(local_path)
# Check the type of fileRDD
print("The file type of fileRDD is", type(fileRDD))
# Check the number of partitions in fileRDD
print("Number of partitions in fileRDD is", fileRDD.getNumPartitions())

The file type of fileRDD is <class 'pyspark.rdd.RDD'>
Number of partitions in fileRDD is 2


In [17]:
# Create a fileRDD_part from file_path with 5 partitions
fileRDD_part = sc.textFile(local_path, minPartitions = 5)
print("Number of partitions after to put a specific partitions number ",fileRDD_part.getNumPartitions())


Number of partitions after to put a specific partitions number  5


In [18]:
numbRDD = sc.parallelize(range(1,11))

In [19]:
numbRDD.take(100)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [20]:
cubedRDD = numbRDD.map(lambda x: x**3)
cubedRDD.take(100)

[1, 8, 27, 64, 125, 216, 343, 512, 729, 1000]

In [21]:
# Collect the results
numbers_all = cubedRDD.collect()
numbers_all

[1, 8, 27, 64, 125, 216, 343, 512, 729, 1000]

In [22]:
#Practice SC functions do contexto
#Create map() transformation to cube numbers
numbRDD = sc.parallelize(range(1,11))
cubedRDD = numbRDD.map(lambda x: x**3)
# OBS: The result can´t be show with function show(), in this case we need use take()
# cubedRDD.take(5)

# Collect the results
numbers_all = cubedRDD.collect()

# Print the numbers from numbers_all
for numb in numbers_all:
 print(numb)

1
8
27
64
125
216
343
512
729
1000


In [23]:
# Filter the fileRDD to select lines with Spark keyword
fileRDD_filter = fileRDD.filter(lambda line: '100' in line)

# How many lines are there in fileRDD?
print("The total number of lines with the keyword Spark is", fileRDD_filter.count())

# Print the first four lines of fileRDD
for line in fileRDD_filter.take(6): 
  print(line)

The total number of lines with the keyword Spark is 4
51.0,1.0,3.0,100.0,222.0,0.0,0.0,143.0,1.0,1.2,2.0,0.0,3.0,0
58.0,0.0,4.0,100.0,248.0,0.0,2.0,122.0,0.0,1.0,2.0,0.0,3.0,0
67.0,1.0,4.0,100.0,299.0,0.0,2.0,125.0,1.0,0.9,2.0,2.0,3.0,3
58.0,1.0,4.0,100.0,234.0,0.0,0.0,156.0,0.0,0.1,1.0,1.0,7.0,2


In [24]:
##########################
# Trabalhando com RDD Pairs transformations
##########################
# Create PairRDD Rdd with key value pairs
Rdd = sc.parallelize([(1,2),(3,4),(3,6),(4,5)])

# Apply reduceByKey() operation on Rdd
Rdd_Reduced = Rdd.reduceByKey(lambda x, y: x + y)

# Iterate over the result and print the output
for num in Rdd_Reduced.collect(): 
  print("Key {} has {} Counts".format(num[0], num[1]))



Key 1 has 2 Counts
Key 3 has 10 Counts
Key 4 has 5 Counts


In [25]:
# Sort the reduced RDD with the key by descending order
Rdd_Reduced_Sort = Rdd_Reduced.sortByKey(ascending=False)

# Iterate over the result and retrieve all the elements of the RDD
for num in Rdd_Reduced_Sort.collect():
  print("Key {} has {} Counts".format(num[0], num[1]))

Key 4 has 5 Counts
Key 3 has 10 Counts
Key 1 has 2 Counts


In [26]:
# Count the unique keys
total = Rdd.countByKey()

# What is the type of total?
print("The type of total is", type(total))

# Iterate over the total and print the output
for k, v in total.items(): 
  print("key", k, "has", v, "counts")

The type of total is <class 'collections.defaultdict'>
key 1 has 1 counts
key 3 has 2 counts
key 4 has 1 counts


In [27]:
#######################
# Managment with dataframe
#######################

# Select _c0,_c1,_c2 of birth columns
df_sub = df.select('sex', 'trestbps', 'slope')
df_sub.show(5)
df_sub.count()

+---+--------+-----+
|sex|trestbps|slope|
+---+--------+-----+
|1.0|   145.0|  3.0|
|1.0|   160.0|  2.0|
|1.0|   120.0|  2.0|
|1.0|   130.0|  3.0|
|0.0|   130.0|  1.0|
+---+--------+-----+
only showing top 5 rows



303

In [28]:
df_sub_nodup = df_sub.dropDuplicates()

# Count the number of rows
print("There were {} rows before removing duplicates, and {} rows after removing duplicates"\
      .format(df_sub.count(), df_sub_nodup.count()))

There were 303 rows before removing duplicates, and 125 rows after removing duplicates


In [29]:
# Filter people_df to select Nunavut 
people_df_Nunavut = people_df.filter(people_df._c7 == "Nunavut")

# Filter people_df to select Northwest Territories
people_df_nt = people_df.filter(people_df._c7 == "Northwest Territories")

# Count the number of rows 
print("There are {} rows in the people_df_female DataFrame and {} rows in the people_df_male DataFrame"\
      .format(people_df_Nunavut.count(), people_df_nt.count()))

NameError: name 'people_df' is not defined

In [30]:
# 1=Male, 0=Female
df_sub_male = df_sub.filter(df_sub.sex == 1)
df_sub_female = df_sub.filter(df_sub.sex == 0)

print(f"There are {df_sub_male.count()} males e {df_sub_female.count()} females in the dataframe")

There are 206 males e 97 females in the dataframe


In [31]:
df.show(5)

+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
| age|sex| cp|trestbps| chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|num|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
|63.0|1.0|1.0|   145.0|233.0|1.0|    2.0|  150.0|  0.0|    2.3|  3.0|0.0| 6.0|  0|
|67.0|1.0|4.0|   160.0|286.0|0.0|    2.0|  108.0|  1.0|    1.5|  2.0|3.0| 3.0|  2|
|67.0|1.0|4.0|   120.0|229.0|0.0|    2.0|  129.0|  1.0|    2.6|  2.0|2.0| 7.0|  1|
|37.0|1.0|3.0|   130.0|250.0|0.0|    0.0|  187.0|  0.0|    3.5|  3.0|0.0| 3.0|  0|
|41.0|0.0|2.0|   130.0|204.0|0.0|    2.0|  172.0|  0.0|    1.4|  1.0|0.0| 3.0|  0|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
only showing top 5 rows



In [32]:
# Create temporary table "df_base"
df.createOrReplaceTempView("df_base")

In [33]:
df.columns

['age',
 'sex',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal',
 'num']

In [34]:
# Contrução da query
query = '''SELECT age, 
                count(Age) AS qtd
            FROM df_base
            GROUP BY age
            Order BY qtd DESC
            LIMIT 100'''

df_age_agr = spark.sql(query)
df_age_agr.show(5)

+----+---+
| age|qtd|
+----+---+
|58.0| 19|
|57.0| 17|
|54.0| 16|
|59.0| 14|
|52.0| 13|
+----+---+
only showing top 5 rows



In [35]:
df.filter(col('sex') == 1).select(col('cp').alias('new_cp'),col('ca').alias('new_ca')).show(10)

+------+------+
|new_cp|new_ca|
+------+------+
|   1.0|   0.0|
|   4.0|   3.0|
|   4.0|   2.0|
|   3.0|   0.0|
|   2.0|   0.0|
|   4.0|   1.0|
|   4.0|   0.0|
|   4.0|   0.0|
|   3.0|   1.0|
|   2.0|   0.0|
+------+------+
only showing top 10 rows



In [36]:
# Analisando dados invalidao "ca"
df.groupBy("ca").agg(count("*")).orderBy("ca").show()

+---+--------+
| ca|count(1)|
+---+--------+
|0.0|     176|
|1.0|      65|
|2.0|      38|
|3.0|      20|
|  ?|       4|
+---+--------+



In [37]:
# Analisando dados invalidao "ca"
df.groupBy("thal").agg(count("*")).orderBy("thal").show()

+----+--------+
|thal|count(1)|
+----+--------+
| 3.0|     166|
| 6.0|      18|
| 7.0|     117|
|   ?|       2|
+----+--------+



In [38]:
# Deletando linhas com erro
df_clean = df.filter(col("ca")!="?").filter(col("thal")!='?')

In [39]:
df_clean.count()

297

In [40]:
df_clean_format = df_clean.withColumn("ca", col("ca").cast("double")) \
        .withColumn("thal",col("thal").cast("double"))

In [41]:
df_clean_format.printSchema()

root
 |-- age: double (nullable = true)
 |-- sex: double (nullable = true)
 |-- cp: double (nullable = true)
 |-- trestbps: double (nullable = true)
 |-- chol: double (nullable = true)
 |-- fbs: double (nullable = true)
 |-- restecg: double (nullable = true)
 |-- thalach: double (nullable = true)
 |-- exang: double (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slope: double (nullable = true)
 |-- ca: double (nullable = true)
 |-- thal: double (nullable = true)
 |-- num: integer (nullable = true)



### Predictive model

In [255]:
!pip install xgboost==1.6.*

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 MB 16.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.0.3
    Uninstalling xgboost-2.0.3:
      Successfully uninstalled xgboost-2.0.3


In [256]:
#Create model with XGBoost Livery
# !pip install xgboost==1.6.8
from xgboost.spark import SparkXGBClassifier
# from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.sql.functions import rand, round
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline

In [257]:
 xgb_classifier = SparkXGBClassifier(num_workers=2,
                                           use_gpu=True,
                                           max_depth=5)

In [258]:
# def create_spark_df(X, y):
#     return spark.createDataFrame(
#         spark.sparkContext.parallelize(
#             [(Vectors.dense(features), float(label)) for features, label in zip(X, y)]
#         ),
#         ["features", "label"],
#     )

In [259]:
df_clean_format.show(10)

+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
| age|sex| cp|trestbps| chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|num|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
|63.0|1.0|1.0|   145.0|233.0|1.0|    2.0|  150.0|  0.0|    2.3|  3.0|0.0| 6.0|  0|
|67.0|1.0|4.0|   160.0|286.0|0.0|    2.0|  108.0|  1.0|    1.5|  2.0|3.0| 3.0|  2|
|67.0|1.0|4.0|   120.0|229.0|0.0|    2.0|  129.0|  1.0|    2.6|  2.0|2.0| 7.0|  1|
|37.0|1.0|3.0|   130.0|250.0|0.0|    0.0|  187.0|  0.0|    3.5|  3.0|0.0| 3.0|  0|
|41.0|0.0|2.0|   130.0|204.0|0.0|    2.0|  172.0|  0.0|    1.4|  1.0|0.0| 3.0|  0|
|56.0|1.0|2.0|   120.0|236.0|0.0|    0.0|  178.0|  0.0|    0.8|  1.0|0.0| 3.0|  0|
|62.0|0.0|4.0|   140.0|268.0|0.0|    2.0|  160.0|  0.0|    3.6|  3.0|2.0| 3.0|  3|
|57.0|0.0|4.0|   120.0|354.0|0.0|    0.0|  163.0|  1.0|    0.6|  1.0|0.0| 3.0|  0|
|63.0|1.0|4.0|   130.0|254.0|0.0|    2.0|  147.0|  0.0|    1.4|  2.0|1.0| 7.0|  2|
|53.

In [260]:
 # -- Value 0: < 50% diameter narrowing
 #    -- Value 1: > 50% diameter narrowing
# df.select("num").distinct().show()
df_clean_format.groupBy("num").agg(count("*").alias("Quantidade")).show()
df_clean_format.groupBy("num").agg(count("*").alias("Quantidade")).show()

+---+----------+
|num|Quantidade|
+---+----------+
|  1|        54|
|  3|        35|
|  4|        13|
|  2|        35|
|  0|       160|
+---+----------+

+---+----------+
|num|Quantidade|
+---+----------+
|  1|        54|
|  3|        35|
|  4|        13|
|  2|        35|
|  0|       160|
+---+----------+



In [261]:
##Create the label
# 1 if num > 0
# 0 if num = 0
condicao_1 = (df.num > 0)
condicao_2 = (df.num == 0)
df_with_label = df_clean_format.withColumn("num_label",when(condicao_1, 1.0).otherwise(0.0)).drop("num")

In [262]:
df_with_label.show()

+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---------+
| age|sex| cp|trestbps| chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|num_label|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---------+
|63.0|1.0|1.0|   145.0|233.0|1.0|    2.0|  150.0|  0.0|    2.3|  3.0|0.0| 6.0|      0.0|
|67.0|1.0|4.0|   160.0|286.0|0.0|    2.0|  108.0|  1.0|    1.5|  2.0|3.0| 3.0|      1.0|
|67.0|1.0|4.0|   120.0|229.0|0.0|    2.0|  129.0|  1.0|    2.6|  2.0|2.0| 7.0|      1.0|
|37.0|1.0|3.0|   130.0|250.0|0.0|    0.0|  187.0|  0.0|    3.5|  3.0|0.0| 3.0|      0.0|
|41.0|0.0|2.0|   130.0|204.0|0.0|    2.0|  172.0|  0.0|    1.4|  1.0|0.0| 3.0|      0.0|
|56.0|1.0|2.0|   120.0|236.0|0.0|    0.0|  178.0|  0.0|    0.8|  1.0|0.0| 3.0|      0.0|
|62.0|0.0|4.0|   140.0|268.0|0.0|    2.0|  160.0|  0.0|    3.6|  3.0|2.0| 3.0|      1.0|
|57.0|0.0|4.0|   120.0|354.0|0.0|    0.0|  163.0|  1.0|    0.6|  1.0|0.0| 3.0|      0.0|
|63.0|1.0|4.0|   130.

In [263]:
## Particionamento em teste e treino
random_state = 67
df_train, df_test = df_with_label.randomSplit(weights=[0.8,0.2], seed=random_state)

# df_label_train_X = df_train.drop("num_label")
# df_label_train_y = df_train.select("num_label")

# df_label_test_X = df_test.drop("num_label")
# df_label_test_y = df_test.select("num_label")

# print(f"""tamanho do dataset de treino features: {(df_label_train_X.count(),len(df_label_train_X.columns))}
#         \ntamanho do dataset de treino label: {(df_label_train_y.count(),len(df_label_train_y.columns))}""")
# print("-"*50)
# print(f"""\ntamanho do dataset de test features: {(df_label_test_X.count(),len(df_label_test_X.columns))}
#         \ntamanho do dataset de test label: {(df_label_test_y.count(),len(df_label_test_y.columns))}""")
# # print(f"\n tamanho do dataset de teste features: {}, label: {}")

In [264]:
df_train.printSchema()

root
 |-- age: double (nullable = true)
 |-- sex: double (nullable = true)
 |-- cp: double (nullable = true)
 |-- trestbps: double (nullable = true)
 |-- chol: double (nullable = true)
 |-- fbs: double (nullable = true)
 |-- restecg: double (nullable = true)
 |-- thalach: double (nullable = true)
 |-- exang: double (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slope: double (nullable = true)
 |-- ca: double (nullable = true)
 |-- thal: double (nullable = true)
 |-- num_label: double (nullable = false)



Diferença entre Dense vetor e Sparce Vetor na saida do transform

- **DenseVector**: É usado quando a maioria dos elementos do vetor é diferente de zero. Ele armazena todos os elementos do vetor sequencialmente, independentemente de serem zero ou não.
- **SparseVector**: É usado quando a maioria dos elementos do vetor é zero. Ele armazena apenas os elementos não nulos e suas posições no vetor, economizando espaço de armazenamento.

Note. No seu caso, alguns vetores são densos (DenseVector) e outros são esparsos (SparseVector). Isso ocorre porque o VectorAssembler detecta automaticamente a densidade dos dados e escolhe a representação mais eficiente para cada vetor.

In [265]:
## TRanformando features num vetor denso
def vetorization_df(data):
    colunas_entrada = ['age','sex','cp','trestbps','chol','fbs','restecg',
                       'thalach','exang','oldpeak','slope','ca','thal']
    assembler = VectorAssembler(inputCols=colunas_entrada, outputCol="features")
    df_com_vetor_denso = assembler.transform(data)
    return df_com_vetor_denso
    # return df_com_vetor_denso.select(col("features"))

In [266]:
df_train_vetor = vetorization_df(df_train).select(col("features"),col("num_label").alias("label"))

df_test_vetor = vetorization_df(df_test).select(col("features"),col("num_label").alias("label"))

In [267]:
#Treinamento do modelo
xgb_classifier_model = xgb_classifier.fit(df_train_vetor)

2024-02-14 18:26:47,902 WARNING SparkXGBClassifier: _validate_gpu_params You have enabled GPU in spark local mode. Please make sure your local node has at least 2 GPUs
2024-02-14 18:26:48,073 INFO XGBoost-PySpark: _fit Running xgboost-1.6.2 on 2 workers with
	booster params: {'objective': 'binary:logistic', 'device': 'cpu', 'max_depth': 5, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Running xgboost-1.6.2 on 2 workers with
	booster params: {'objective': 'binary:logistic', 'device': 'cpu', 'max_depth': 5, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-02-14 18:26:48,148 INFO SparkXGBClassifier: _skip_stage_level_scheduling Stage-level scheduling in xgboost requires spark standalone or local-cluster mode
INFO:SparkXGBClassifier:Stage-level scheduling in xgboost requires spark standalon

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Could not recover from a failed barrier ResultStage. Most recent failure reason: Stage failed because barrier task ResultTask(1665, 0) finished unsuccessfully.
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1231, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 912, in read_udfs
    arg_offsets, udf = read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=0)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/pyspark.z

In [158]:
transformed_df_test_vetor = xgb_classifier_model.transform(df_test_vetor)

In [160]:
transformed_df_test_vetor.show()

+--------------------+-----+--------------------+----------+--------------------+
|            features|label|       rawPrediction|prediction|         probability|
+--------------------+-----+--------------------+----------+--------------------+
|[34.0,1.0,1.0,118...|  0.0|[6.14322900772094...|       0.0|[0.99785661697387...|
|[35.0,1.0,4.0,126...|  1.0|[-3.9950373172760...|       1.0|[0.01807403564453...|
|(13,[0,2,3,4,7,10...|  0.0|[8.07112503051757...|       0.0|[0.99968767166137...|
|[37.0,1.0,3.0,130...|  0.0|[2.18687820434570...|       0.0|[0.89906495809555...|
|(13,[0,2,3,4,7,10...|  0.0|[6.39008617401123...|       0.0|[0.99832469224929...|
|[41.0,0.0,2.0,130...|  0.0|[7.96419095993042...|       0.0|[0.99965244531631...|
|[41.0,1.0,2.0,110...|  0.0|[1.53712546825408...|       0.0|[0.82304644584655...|
|[41.0,1.0,2.0,135...|  0.0|[2.32624101638793...|       0.0|[0.91102713346481...|
|[41.0,1.0,3.0,130...|  0.0|[5.34667444229126...|       0.0|[0.99525862932205...|
|[42.0,1.0,3.0,1

In [164]:
## Metrica do resultado para classificação
classifier_evaluator = MulticlassClassificationEvaluator(metricName="f1")
print(f"classifier f1={classifier_evaluator.evaluate(transformed_df_test_vetor)}")

classifier f1=0.651299464542249


In [197]:
## Criando um segundo modelo com a coluna de ValidationIndicatorCol a mais

df_train_vetor_2 = df_train_vetor.withColumn("ValidationIndicatorCol",rand(random_state) >0.7)
df_train_vetor_2.show(5)

+--------------------+-----+----------------------+
|            features|label|ValidationIndicatorCol|
+--------------------+-----+----------------------+
|[29.0,1.0,2.0,130...|  0.0|                  true|
|[34.0,0.0,2.0,118...|  0.0|                 false|
|[35.0,0.0,4.0,138...|  0.0|                  true|
|[35.0,1.0,2.0,122...|  0.0|                 false|
|[35.0,1.0,4.0,120...|  1.0|                 false|
+--------------------+-----+----------------------+
only showing top 5 rows



**validationIndicatorCol:** Esta opção é usada para especificar a coluna de indicação de validação. Essa coluna é usada para identificar os registros que devem ser incluídos no conjunto de validação durante o treinamento do modelo. Geralmente, você divide seu conjunto de dados em conjuntos de treinamento e validação para avaliar o desempenho do modelo durante o treinamento. A coluna de indicação de validação é usada para indicar quais registros pertencem ao conjunto de validação. Se fornecido, o modelo usará essa coluna durante o treinamento para separar os dados em conjuntos de treinamento e validação. Se não for fornecido, o modelo usará uma divisão aleatória padrão.

**weightCol:** Esta opção é usada para especificar a coluna que contém os pesos associados a cada amostra. Os pesos são usados para fornecer importância diferenciada às amostras durante o treinamento do modelo. Por exemplo, se algumas amostras são mais importantes ou mais representativas do que outras, você pode atribuir pesos mais altos a essas amostras para que o modelo leve isso em consideração durante o treinamento. Se não for fornecido, todos os pesos são considerados iguais.

**base_margin_col:** Pode usar essa coluna como base_margin_col no modelo XGBoost para fornecer ao modelo uma pontuação inicial para cada cliente, refletindo sua confiança. Durante o treinamento, o modelo então ajustaria esses valores de margem base para otimizar a previsão

In [191]:
#Invocar momdelo indicando coluna de validação do indicador
xgb_classifier2 = SparkXGBClassifier(
    max_depth=5, validation_indicator_col="ValidationIndicatorCol"
)

In [176]:
# Treinamento novamente do modelo
xgb_classifier_model2 = xgb_classifier2.fit(df_train_vetor_2)
transformed_df_train_vetor_2 = xgb_classifier_model2.transform(df_train_vetor_2)
transformed_df_train_vetor_2.show()
print(
    f"classifier2 f1={classifier_evaluator.evaluate(transformed_df_train_vetor_2)}"
)

2024-02-13 22:48:28,721 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 1 workers with
	booster params: {'objective': 'binary:logistic', 'device': 'cpu', 'max_depth': 5, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-02-13 22:48:31,793 INFO XGBoost-PySpark: _fit Finished xgboost training!


+--------------------+-----+----------------------+--------------------+----------+--------------------+
|            features|label|ValidationIndicatorCol|       rawPrediction|prediction|         probability|
+--------------------+-----+----------------------+--------------------+----------+--------------------+
|[29.0,1.0,2.0,130...|  0.0|                 false|[6.67216920852661...|       0.0|[0.99873596429824...|
|[34.0,0.0,2.0,118...|  0.0|                 false|[6.46621751785278...|       0.0|[0.99844729900360...|
|[35.0,0.0,4.0,138...|  0.0|                 false|[5.10124015808105...|       0.0|[0.99394768476486...|
|[35.0,1.0,2.0,122...|  0.0|                 false|[5.42273092269897...|       0.0|[0.99560433626174...|
|[35.0,1.0,4.0,120...|  1.0|                  true|[-4.6494755744934...|       1.0|[0.00947600603103...|
|[38.0,1.0,1.0,120...|  1.0|                  true|[-0.9622104167938...|       1.0|[0.27643585205078...|
|(13,[0,2,3,4,7,10...|  0.0|                 false|[5.7

In [198]:
# Na ValidationIndicatorCol, o valor True são os considerados ara validação e False para treinamento
transformed_df_train_vetor_2.groupBy(col("ValidationIndicatorCol")) \
                            .agg(count("*").alias("Qtd"), 
                                round(count("*")/ transformed_df_train_vetor_2.count()
                                      ,2).alias("Proporção (%)")
                                ).show()

+----------------------+---+-------------+
|ValidationIndicatorCol|Qtd|Proporção (%)|
+----------------------+---+-------------+
|                  true| 68|         0.28|
|                 false|172|         0.72|
+----------------------+---+-------------+



In [201]:
# xgb_classifier_model2.explainParams()
xgb_classifier_model2.extractParamMap()

{Param(parent='SparkXGBClassifier_06594260c6e3', name='enable_sparse_data_optim', doc='This stores the boolean config of enabling sparse data optimization, if enabled, Xgboost DMatrix object will be constructed from sparse matrix instead of dense matrix. This config is disabled by default. If most of examples in your training dataset contains sparse features, we suggest to enable this config.'): False,
 Param(parent='SparkXGBClassifier_06594260c6e3', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='SparkXGBClassifier_06594260c6e3', name='features_cols', doc='feature column names.'): [],
 Param(parent='SparkXGBClassifier_06594260c6e3', name='labelCol', doc='label column name.'): 'label',
 Param(parent='SparkXGBClassifier_06594260c6e3', name='predictionCol', doc='prediction column name.'): 'prediction',
 Param(parent='SparkXGBClassifier_06594260c6e3', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models ou

In [241]:
# Treinameto com validação cruzada e pipeline

xgb_classifier_cross_val = SparkXGBClassifier(
    objetive="binary:logistic",
    # validation_indicator_col="ValidationIndicatorCol"
    num_round=100,
    verbose = 1,
    # max_depth=5, 
    # nthread=1,
)

In [242]:
paramGrid = (ParamGridBuilder()
             .addGrid(xgb_classifier_cross_val.max_depth,[3, 6, 9])
             .addGrid(xgb_classifier_cross_val.learning_rate,[0.1, 0.3,0.5])
             .build())

In [243]:
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")

In [244]:
cv=CrossValidator(estimator = xgb_classifier_cross_val,
                  estimatorParamMaps = paramGrid,
                  evaluator = evaluator,
                  numFolds = 2
                  seed = random_state)

In [245]:
pipeline = Pipeline(stages=[cv])

In [246]:
model = pipeline.fit(df_train_vetor)

2024-02-14 14:29:01,434 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'learning_rate': 0.1, 'max_depth': 3, 'objective': 'binary:logistic', 'objetive': 'binary:logistic', 'num_round': 100, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': 1, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-02-14 14:29:04,424 INFO XGBoost-PySpark: _fit Finished xgboost training!
2024-02-14 14:29:04,755 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'learning_rate': 0.3, 'max_depth': 3, 'objective': 'binary:logistic', 'objetive': 'binary:logistic', 'num_round': 100, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': 1, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-02-14 14:29:07,011 INFO XGBoost-PySpark: _fit Finished xgboost training!
2024-02-14 14:29:07,302 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 1 workers wi

In [250]:
model

PipelineModel_95f100facd4c

In [251]:
model.extractParamMap()

{}